### 1주차 과제 진행
* client.ipynb는 카프카에 요청 위한 코드
* 테스트 시 server.ipynb 실행, client.ipynb 실행 후 색인 진행

1. 제품 listing 전체를 Elasticsearch에 인덱스
2. 제품 인덱스의 ”item_keywords” 와 “product_description”를 업데이트 할 수 있는 API
3. item_keyword와 product_description 정보로 제품을 검색
---
세부사항
Kafka
Elasticsearch
- 로컬환경에서 작동되는 인프라를 구축합니다.(텍스트 전처리는 python의 nltk를 씁니다: https://www.nltk.org/)
- 다량의 업데이트를 지원할 수 있는 시스템 구축을 가정합니다. (Flink: https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/python/overview/)
- language_tag가 en_US 로 된 제품만 인덱스 합니다. (Kafka: https://docs.confluent.io/kafka-clients/python/current/overview.html)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,explode, expr

# SparkSession 생성
spark = SparkSession.builder \
    .appName("Read GZ Compressed JSON Files with PySpark") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .getOrCreate()




24/10/09 23:08:33 WARN Utils: Your hostname, HG380-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
24/10/09 23:08:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/10/09 23:08:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/09 23:08:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import json
import datetime

from kafka import KafkaProducer

def on_success(metadata):
    print(f"Message produced with the offset: {metadata.offset}")

def on_error(error):
    print(f"An error occurred while publishing the message. {error}")

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

message = {
    "item_id": "B083PF2S9N",
    "item_keywords": [
        {
          "language_tag": "en_US",
          "value": "hello world"
        },
        {
          "language_tag": "en_US",
          "value": "good!"
        },
        {
          "language_tag": "ja_JP",
          "value": "セット"
        },
        {
          "language_tag": "en_US",
          "value": "nice!"
        }
    ],
    "product_description": [
        {
            "language_tag": "en_US",
            "value": "test1"
        },
        {
          "language_tag": "ja_JP",
          "value": "収納ボックス 折り畳み ; 収納ボックス セット ; 収納ボックス衣類 ; カラー ボックス 収納 ;"
        },
        {
            "language_tag": "en_US",
            "value": "test2"
        }
    ]
}

future = producer.send("amazon_berkeley_update", message)
future.add_callback(on_success)
future.add_errback(on_error)

# Ensure all messages are sent before exiting
producer.flush()
producer.close()

Message produced with the offset: 4


In [5]:
! curl -XGET "http://localhost:9200/amazon-berkeley/_search" -H "Content-Type: application/json" -d '{"query":{"term":{"item_id.keyword":{"value":"B083PF2S9N"}}},"_source":["item_id","item_keywords","product_description"]}'

{"took":11,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":10.985678,"hits":[{"_index":"amazon-berkeley","_id":"B083PF2S9N","_score":10.985678,"_source":{"item_id":"B083PF2S9N","item_keywords":[{"language_tag":"en_US","value":"hello world"},{"language_tag":"en_US","value":"good!"},{"language_tag":"en_US","value":"nice!"}],"product_description":[{"language_tag":"en_US","value":"test1"},{"language_tag":"en_US","value":"test2"}]}}]}}